# Cider analysis on the full proteome

<a rel="license" href="https://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://licensebuttons.net/l/by-sa/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

Author: Dr Antonia Mey   
Email: antonia.mey@ed.ac.uk

In [1]:
# Imports
import pandas as pd
import numpy as np
import glob 
import os
import urllib
from collections import Counter
import urllib
import json

# import the relevant code
from localcider.sequenceParameters import SequenceParameters

In [2]:
def get_all_data_from_seq(seq):
    ''' This extracts all data needed from cider for a given sequence and returns it as a dictionary
    '''
    SeqOb = SequenceParameters(seq)
    data = {}
    data['N'] = SeqOb.get_length()
    data['seq'] = seq
    data['f-'] = SeqOb.get_fraction_negative()
    data['f+'] = SeqOb.get_fraction_positive()
    data['FCR']= SeqOb.get_FCR()
    data['NCPR'] = SeqOb.get_NCPR()
    data['kappa'] = SeqOb.get_kappa()
    data['Omega'] = SeqOb.get_Omega()
    data['sigma'] = get_sigma(data['f+'], data['f-'])
    data['delta'] = SeqOb.get_delta()
    data['delta max'] = SeqOb.get_deltaMax()
    data['hydropathy'] = SeqOb.get_mean_hydropathy()
    data['Phase plot region'] = int(SeqOb.get_phasePlotRegion())
    data['Hydrophobic count'] = int(get_num_hydrophobic(seq))
    data['Positive count'] = int(SeqOb.get_countPos())
    data['Negative count'] = int(SeqOb.get_countNeg())
    return data

In [3]:
def get_domainn_region_info(data):
    known_things = {}
    known_things['Domain'] = []
    known_things['Region'] = []
    known_things['Motif'] = []
    #known_things['Helix'] = []
    if 'features' in data.keys():
        for d in data['features']:
            #print(d['type'])
            if d['type'] == 'Domain':
                known_things['Domain'].append([d['location']['start']['value'], d['location']['end']['value'], d['description']])
            elif d['type'] == 'Region':
                known_things['Region'].append([d['location']['start']['value'], d['location']['end']['value'], d['description']])
            elif d['type'] == 'Motif':
                known_things['Motif'].append([d['location']['start']['value'], d['location']['end']['value'], d['description']])
            #elif d['type'] == 'Helix':
            #    known_things['Helix'].append([d['location']['start']['value'], d['location']['end']['value'], d['description']])

            
    return known_things       

In [122]:
def get_sigma(f_plus, f_minus):
    '''gets sigma
    Sigma: ⟨σ⟩, or the average sigma value for the entire sequence, where sigma quantifies the charge asymmetry. 
Specifically, σ = (f+-f-)2/(f++f-) where f- and f+ refer to the fraction of negative and positive residues across the entire sequence. See Das & Pappu[2] for more details
    
    '''
    if f_plus == 0 and f_minus==0:
        sigma = np.nan
    else:
        sigma = (f_plus-f_minus)**2/(f_plus+f_minus)
    return sigma

In [123]:
def get_num_hydrophobic(seq):
    ''' counts the number of hydrophobic amino acids (L,V,I,M) in a seq
    '''
    
    I_count = seq.count('I')
    M_count = seq.count('M')
    L_count = seq.count('L')
    V_count = seq.count('V')
    count = I_count+M_count+L_count+V_count
    return count
    

In [61]:
uniprot_ids = glob.glob('unique_ids/*')
protein = uniprot_ids[20]

In [62]:
f = open(protein)
data = json.load(f)
f.close()

In [63]:
keys = data.keys()
verbose = True
# Primary Accession
uni_id = 'NA'
if 'primaryAccession' in keys:
    uni_id = data['primaryAccession'] 
    if verbose:
         print('uni_id',uni_id)
else: 
    print('Issue: No primary Accession found')

# Genes
gene_name = 'NA'
if 'genes' in keys:
    gene_name = data['genes'][0]['geneName']['value']
    if verbose:
        print('gene_name',gene_name)
else:
    print('Issue: No gene found found')

# getting the nucleotide sequenceID
ref_seq_id = 'NA'
if 'uniProtKBCrossReferences' in keys:
    for db in data['uniProtKBCrossReferences']:
        if db['database'] == "RefSeq":
            if '.' in db['properties'][0]['value']:
                ref_seq_id = db['properties'][0]['value'].split('.')[0]
            else:
                ref_seq_id = db['properties'][0]['value']
if verbose:
    print('ref_seq_id',ref_seq_id)

# getting sequence info
sequence = 'NA'
seq_length = 0
if 'sequence' in keys:
    sequence = data['sequence']['value']
    if verbose:
        print('sequence',sequence)

    seq_length = data['sequence']['length']
    if seq_length != len(sequence):
        print(f"Warning for Uniprot id {uni_prot_id} sequence length recorded is not same as actual sequence")

known_things = get_domainn_region_info(data)


uni_id P46940
gene_name IQGAP1
ref_seq_id NM_003870
sequence MSAADEVDGLGVARPHYGSVLDNERLTAEEMDERRRQNVAYEYLCHLEEAKRWMEACLGEDLPPTTELEEGLRNGVYLAKLGNFFSPKVVSLKKIYDREQTRYKATGLHFRHTDNVIQWLNAMDEIGLPKIFYPETTDIYDRKNMPRCIYCIHALSLYLFKLGLAPQIQDLYGKVDFTEEEINNMKTELEKYGIQMPAFSKIGGILANELSVDEAALHAAVIAINEAIDRRIPADTFAALKNPNAMLVNLEEPLASTYQDILYQAKQDKMTNAKNRTENSERERDVYEELLTQAEIQGNINKVNTFSALANIDLALEQGDALALFRALQSPALGLRGLQQQNSDWYLKQLLSDKQQKRQSGQTDPLQKEELQSGVDAANSAAQQYQRRLAAVALINAAIQKGVAEKTVLELMNPEAQLPQVYPFAADLYQKELATLQRQSPEHNLTHPELSVAVEMLSSVALINRALESGDVNTVWKQLSSSVTGLTNIEEENCQRYLDELMKLKAQAHAENNEFITWNDIQACVDHVNLVVQEEHERILAIGLINEALDEGDAQKTLQALQIPAAKLEGVLAEVAQHYQDTLIRAKREKAQEIQDESAVLWLDEIQGGIWQSNKDTQEAQKFALGIFAINEAVESGDVGKTLSALRSPDVGLYGVIPECGETYHSDLAEAKKKKLAVGDNNSKWVKHWVKGGYYYYHNLETQEGGWDEPPNFVQNSMQLSREEIQSSISGVTAAYNREQLWLANEGLITRLQARCRGYLVRQEFRSRMNFLKKQIPAITCIQSQWRGYKQKKAYQDRLAYLRSHKDEVVKIQSLARMHQARKRYRDRLQYFRDHINDIIKIQAFIRANKARDDYKTLINAEDPPMVVVRKFVHLLDQSDQDFQEELDLMKMREEVITLIRSNQQLENDLNLMDIKIGLLVKNKITLQDVVSHSKKLTK

In [106]:
known_things

{'Domain': [[44, 159, 'Calponin-homology (CH)'],
  [679, 712, 'WW'],
  [745, 774, 'IQ 1'],
  [775, 804, 'IQ 2'],
  [805, 834, 'IQ 3'],
  [835, 864, 'IQ 4'],
  [1004, 1237, 'Ras-GAP']],
 'Region': [[956, 1274, 'C1'], [1276, 1657, 'C2'], [1410, 1448, 'Disordered']],
 'Motif': []}

In [114]:
keys = known_things.keys()
mask = np.zeros(seq_length)
segment_list = []
for k in keys:
    for segement in known_things[k]:
        if len(segement)>0:
            segment_list.append([segement[0]-1,segement[1],segement[2]])
            mask[segement[0]-1:segement[1]] = 1
segment_list.sort()

In [115]:
from itertools import groupby
def false_indices(a):
    i = 0
    for k, g in groupby(a):
        l = len(list(g))
        if not k:
            yield (i,i+l)
        i += l

In [120]:
# now we get the sequence section
for i in range(len(segment_list)):
    seq_chunk = sequence[segment_list[i][0]:segment_list[i][1]]
    #print(segment_list[i][0],segment_list[i][1],seq_chunk)


In [124]:
cider_data = get_all_data_from_seq(seq_chunk)
df = pd.Series(cider_data).to_frame().T

{'N': 39,
 'seq': 'TPATSEQEAEHQRAMQRRAIRDAKTPDKMKKSKSVKEDS',
 'f-': 0.1794871794871795,
 'f+': 0.2564102564102564,
 'FCR': 0.4358974358974359,
 'NCPR': 0.07692307692307693,
 'kappa': 0.24082176888233633,
 'Omega': 0.025890082904646273,
 'sigma': 0.013574660633484155,
 'delta': 0.09073125085333411,
 'delta max': 0.3767568491603627,
 'hydropathy': 2.7923076923076926,
 'Phase plot region': 3,
 'Hydrophobic count': 4,
 'Positive count': 10,
 'Negative count': 7}

In [164]:
vdf = pd.DataFrame(columns=['uniprot_id','gene_name','refseq_id'])
new_data = []
for i in range(100):
    if i%10==0:
        print(f'At entry {i}/{100}')
    protein = uniprot_ids[i]
    f = open(protein)
    data = json.load(f)
    f.close()
    
    # Getting some of the previous info
    keys = data.keys()
    verbose = False
    # Primary Accession
    uni_id = 'NA'
    if 'primaryAccession' in keys:
        uni_id = data['primaryAccession'] 
        if verbose:
             print('uni_id',uni_id)
    else: 
        print('Issue: No primary Accession found')

    # Genes
    gene_name = 'NA'
    if 'genes' in keys:
        gene_name = data['genes'][0]['geneName']['value']
        if verbose:
            print('gene_name',gene_name)
    else:
        print('Issue: No gene found found')

    # getting the nucleotide sequenceID
    ref_seq_id = 'NA'
    if 'uniProtKBCrossReferences' in keys:
        for db in data['uniProtKBCrossReferences']:
            if db['database'] == "RefSeq":
                if '.' in db['properties'][0]['value']:
                    ref_seq_id = db['properties'][0]['value'].split('.')[0]
                else:
                    ref_seq_id = db['properties'][0]['value']
    if verbose:
        print('ref_seq_id',ref_seq_id)

    # getting sequence info
    sequence = 'NA'
    seq_length = 0
    if 'sequence' in keys:
        sequence = data['sequence']['value']
        if verbose:
            print('sequence',sequence)

        seq_length = data['sequence']['length']
        if seq_length != len(sequence):
            print(f"Warning for Uniprot id {uni_prot_id} sequence length recorded is not same as actual sequence")

    known_things = get_domainn_region_info(data)
    
    # Sorting the segments
    keys = known_things.keys()
    mask = np.zeros(seq_length)
    segment_list = []
    for k in keys:
        for segement in known_things[k]:
            if len(segement)>0:
                segment_list.append([segement[0]-1,segement[1],segement[2]])
                mask[segement[0]-1:segement[1]] = 1
    segment_list.sort()
    
    # finding unassigned segments
    unassigned = list(false_indices(mask))
    for u in unassigned:
        segment_list.append([u[0], u[1], 'unassigned'])
    segment_list.sort()
    
    ## Doing the cider analsyis
    for i in range(len(segment_list)):
        
        seq_chunk = sequence[segment_list[i][0]:segment_list[i][1]]
        cider_data = get_all_data_from_seq(seq_chunk)
        columns = {'uniprot_id':uni_id,'gene_name':gene_name,'refseq_id':ref_seq_id, 'annotation':segment_list[i][2]}
        concat_dic = {}
        concat_dic.update(columns)
        concat_dic.update(cider_data)
        new_data.append(concat_dic)

At entry 0/100
At entry 10/100
At entry 20/100
At entry 30/100
At entry 40/100
At entry 50/100
At entry 60/100
At entry 70/100
At entry 80/100
Issue: No gene found found
At entry 90/100


In [165]:
new_columns = pd.DataFrame(new_data)  

In [166]:
new_columns

,uniprot_id,gene_name,refseq_id,annotation,N,seq,f-,f+,FCR,NCPR,kappa,Omega,sigma,delta,delta max,hydropathy,Phase plot region,Hydrophobic count,Positive count,Negative count
0,Q96GA3,LTV1,NM_032860,unassigned,22,MPHRKKKPFIEKKKAVSFHLVH,0.045455,0.318182,0.363636,0.272727,0.425427,0.359966,0.204545,0.076269,0.179277,3.636364,3,5,7,1
1,Q96GA3,LTV1,NM_032860,Disordered,32,RSQRDPLAADESAPQRVLLPTQKIDNEERRAE,0.218750,0.187500,0.406250,-0.031250,0.123860,0.036508,0.002404,0.042293,0.341457,3.103125,3,5,6,7
2,Q96GA3,LTV1,NM_032860,unassigned,19,QRKYGVFFDDDYDYLQHLK,0.210526,0.157895,0.368421,-0.052632,0.665697,0.164810,0.007519,0.159463,0.239543,3.268421,3,3,3,4
3,Q96GA3,LTV1,NM_032860,Disordered,26,EPSGPSELIPSSTFSAHNRREEKEET,0.230769,0.115385,0.346154,-0.115385,0.076693,0.403497,0.038462,0.017630,0.229882,3.026923,2,2,3,6
4,Q96GA3,LTV1,NM_032860,unassigned,69,LVIPSTGIKLPSSVFASEFEEDVGLLNKAAPVSGPRLDFDPDIVAA...,0.246377,0.043478,0.289855,-0.202899,0.387080,0.112137,0.142029,0.073485,0.189845,4.492754,2,18,3,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,P60608,ERVFC1-1,NA,CX6CC,9,CLFLQEECC,0.222222,0.000000,0.222222,-0.222222,0.603249,0.312193,0.222222,0.019485,0.032299,5.322222,1,2,0,2
496,Q9C0B5,ZDHHC5,XM_011544901,unassigned,103,MPAESGKRFKPSKYVPVSAAAIFLVGATTLFFAFTCPGLSLYVSPA...,0.106796,0.097087,0.203883,-0.009709,0.232912,0.431847,0.000462,0.041226,0.177004,4.803883,1,26,10,11
497,Q9C0B5,ZDHHC5,XM_011544901,DHHC,51,KWCATCRFYRPPRCSHCSVCDNCVEEFDHHCPWVNNCIGRRNYRYF...,0.078431,0.137255,0.215686,0.058824,0.455520,0.233759,0.016043,0.069942,0.153544,4.158824,1,7,7,4
498,Q9C0B5,ZDHHC5,XM_011544901,unassigned,134,SLTAHIMGVFGFGLLYVLYHIEELSGVRTAVTMAVMCVAGLFFIPV...,0.052239,0.104478,0.156716,0.052239,0.215688,0.408543,0.017413,0.027919,0.129439,4.811194,1,40,14,7


In [167]:
new_columns.to_csv('cider/example_output.csv')

## Old stuff may not be needed

In [4]:
def get_alpha_helix_length_and_location(secondary_struc, min_length=8):
    helix_regions = []
    counter = 0
    curr_helix = []
    indexes = [i for i, x in enumerate(list(secondary_struc)) if x == 'H']
    for i in range(len(indexes)-1):
        difference = indexes[i+1]-indexes[i]
        if difference == 1:
            curr_helix.append(indexes[i])
            if i == len(indexes)-2:
                if len(curr_helix)>=min_length-1:
                    curr_helix.append(indexes[i+1])
                    helix_regions.append(curr_helix)
        else:
            curr_helix.append(indexes[i])
            if len(curr_helix)>=min_length-1:
                helix_regions.append(curr_helix)
            curr_helix = []
    return helix_regions

In [5]:
def count_serines(sequence):
    n_serine = sequence.count('S')
    if n_serine >=2:
        return 1
    else:
        return 0

In [6]:
def filter_helices(helix_index_list, known_things, sequence_length, discard_list= ['bHLH', 'Leucine-zipper'],overlap_threshold =4):
    '''
    Parameters:
    -----------
    helix_index_list : list
        2D list containing arrays of indexes where alpha helixes are
    known_things : dictionary
        extracted information from uniprot IDs around regions, domains and motives
    discard_list : list of Strings
        list that contains the identifiers to discard
    
    Returns:
    --------
    helix_index_list : list
        filtered list with correct helix indexes without the overlap
    Algorithm description:
    
    - generate a boolean array of all False of length of the sequence
    - for each annotation we want to check:
    - Add True between start and end of domains/regions we want to check in the boolean array
    - Then loop over helix list creating a boolean array of length sequence for each Helix section
    - Use logic and to compare regions boolean array with helix boolean array. 
    - If the number of over lap of Trues is larger than theshold x, remove this helix chunch from list and don't write to spreadsheet.
    '''
    
    bool_array =  np.zeros(sequence_length, dtype=bool)
    # alpha_helix_index_list = get_alpha_helix_length_and_location(sec_struc_output, min_length=7)
    helix_list_remove_index = []
    for key in known_things.keys():
        for entry in known_things[key]:
            bool_array =  np.zeros(sequence_length, dtype=bool)
            if entry[-1] in discard_list:
                bool_array[entry[0]-1:entry[1]] = True
            index = 0
            for a in helix_index_list:
                curr_helix =  np.zeros(sequence_length, dtype=bool)
                curr_helix[a] = True
                overalp = (np.sum(np.logical_and(bool_array, curr_helix)))
                if overalp>overlap_threshold:
                    helix_list_remove_index.append(index)
                index = index+1
    unique_idx = np.unique(helix_list_remove_index)
    for index in sorted(unique_idx, reverse=True):
        del helix_index_list[index]
    return helix_index_list

In [7]:
def annotating_helices(helix_index_list, known_things, sequence_length,overlap_threshold =4):
    '''
    Parameters:
    -----------
    helix_index_list : list
        2D list containing arrays of indexes where alpha helixes are
    known_things : dictionary
        extracted information from uniprot IDs around regions, domains and motives
    discard_list : list of Strings
        list that contains the identifiers to discard
    
    Returns:
    --------
    helix_index_list : list
        filtered list with correct helix indexes without the overlap
    Algorithm description:
    
    - generate a boolean array of all False of length of the sequence
    - for each annotation we want to check:
    - Add True between start and end of domains/regions we want to check in the boolean array
    - Then loop over helix list creating a boolean array of length sequence for each Helix section
    - Use logic and to compare regions boolean array with helix boolean array. 
    - If the number of over lap of Trues is larger than theshold x, remove this helix chunch from list and don't write to spreadsheet.
    '''
    annotations = [""]*len(helix_index_list)
    for key in known_things.keys():
        for entry in known_things[key]:
            # print(f'checking entry {entry}')
            bool_array =  np.zeros(sequence_length, dtype=bool)
            if entry[0] is None:
                continue
            elif entry[1] is None:
                continue
            bool_array[entry[0]-1:entry[1]] = True
            index = 0
            
            for a in helix_index_list:
                curr_helix =  np.zeros(sequence_length, dtype=bool)
                curr_helix[a] = True
                overalp = (np.sum(np.logical_and(bool_array, curr_helix)))
                if overalp>overlap_threshold:
                    # print(f'annotation found {entry[-1]}')
                    annotations[index] = annotations[index] +key+':'+entry[-1]+";"
                index = index+1
    return annotations

In [8]:
def get_discrad_list(domain_f_name, region_f_name):
    # Getting domains
    domain_filter = []
    # This should go into a try thing!
    f = open(domain_f_name)
    f_content = f.readlines()
    f.close()

    for l in f_content:
        domain_filter.append(l.strip()) 
        
    # Getting regions
    region_filter = []
    f = open(region_f_name)
    f_content = f.readlines()
    f.close()

    for l in f_content:
        region_filter.append(l.strip())
    discard_list = np.hstack((region_filter,domain_filter))
    return discard_list

In [9]:
def generate_table_rows_for_uniprot_id(uni_prot_id, df, work_dir = 'temp', verbose = False, annotate = False):
    # print('Working on uniprot ID', uni_prot_id)
    sec_struc_output = np.load('dssp_info/'+uni_prot_id+'.npy')
    # print('got secondary structure')
    #print(sec_struc_output)
    
    # Process uniprot file
    f = open('unique_ids/'+uni_prot_id+'.json')
    data = json.load(f)
    f.close()
    
    # print('read json file successfully')
    
    # Some basics:
    # checking keys
    keys = data.keys()
    
    # Primary Accession
    uni_id = 'NA'
    if 'primaryAccession' in keys:
        uni_id = data['primaryAccession'] 
        if verbose:
             print('uni_id',uni_id)
    else: 
        print('Issue: No primary Accession found')
    
    # Genes
    gene_name = 'NA'
    if 'genes' in keys:
        gene_name = data['genes'][0]['geneName']['value']
        if verbose:
            print('gene_name',gene_name)
    else:
        print('Issue: No gene found found')
    
    # getting the nucleotide sequenceID
    ref_seq_id = 'NA'
    if 'uniProtKBCrossReferences' in keys:
        for db in data['uniProtKBCrossReferences']:
            if db['database'] == "RefSeq":
                if '.' in db['properties'][0]['value']:
                    ref_seq_id = db['properties'][0]['value'].split('.')[0]
                else:
                    ref_seq_id = db['properties'][0]['value']
    if verbose:
        print('ref_seq_id',ref_seq_id)
    
    # getting sequence info
    sequence = 'NA'
    seq_length = 0
    if 'sequence' in keys:
        sequence = data['sequence']['value']
        if verbose:
            print('sequence',sequence)
    
        seq_length = data['sequence']['length']
        if seq_length != len(sequence):
            print(f"Warning for Uniprot id {uni_prot_id} sequence length recorded is not same as actual sequence")

    if seq_length == len(sec_struc_output):

        

        # Now lets get the helices:
        alpha_helix_index_list = get_alpha_helix_length_and_location(sec_struc_output, min_length=8)

        # Get the information of what domains and regions exist in the uniport id file:
        known_things = get_domainn_region_info(data)

        discard_list = ['PUM-HD', 'HEAT', 'ARM', 'bHLH', 'Leucine-zipper']
        discard_list = get_discrad_list('data/domain.csv', 'data/region_filter.csv')
        discard_list = []

        # Missing: Filter the alpha_helix_index_list
        filtered_helix_list=filter_helices(alpha_helix_index_list, known_things, seq_length, discard_list=discard_list,overlap_threshold =4)

        # annotate helices:
        if annotate:
            filtered_helix_list = alpha_helix_index_list
            annotations = annotating_helices(filtered_helix_list, known_things, seq_length,overlap_threshold =4)

        for i in range(len(filtered_helix_list)):
            firstAA_position_in_HELIDR = filtered_helix_list[i][0]+1
            lastAA_position_in_HELIDR = filtered_helix_list[i][-1]+1
            HELIDR_seq = sequence[filtered_helix_list[i][0]:filtered_helix_list[i][-1]+1]
            down_stream_seq = ''
            up_stream_seq = ''
            if filtered_helix_list[i][0]-10 >= 0 and filtered_helix_list[i][-1]+11 < len(sequence):
                # Note the +2 here does not include the last helix AA. 
                # We have to make sure here that we check the arrays are not out of bounds!
                down_stream_seq = sequence[filtered_helix_list[i][-1]+1:filtered_helix_list[i][-1]+11]
                up_stream_seq = sequence[filtered_helix_list[i][0]-10:filtered_helix_list[i][0]]
            elif alpha_helix_index_list[i][0]-10 <= 0:
                # Do we want a shorter version? 
                down_stream_seq = sequence[filtered_helix_list[i][-1]+1:filtered_helix_list[i][-1]+11]
                up_stream_seq = sequence[0:filtered_helix_list[i][0]]
            elif alpha_helix_index_list[i][-1]+11 > len(sequence):
                down_stream_seq = sequence[filtered_helix_list[i][-1]+2:len(sequence)+1]
                up_stream_seq = sequence[filtered_helix_list[i][0]-10:filtered_helix_list[i][0]]

            # This may need fixing if we have shorter upstream and downstream strings
            if len(down_stream_seq)==10 and len(up_stream_seq)==10: 
                Two_S5P_down = count_serines(down_stream_seq[:5])
                Two_S5P_up = count_serines(up_stream_seq[5:])
                # NOW we assemble the row:
                if annotate:
                    new_row = [uni_id,gene_name,ref_seq_id,firstAA_position_in_HELIDR,lastAA_position_in_HELIDR,up_stream_seq,HELIDR_seq,
                          down_stream_seq,Two_S5P_up,Two_S5P_down,'','','','','','','','','',annotations[i]]
                else:
                    new_row = [uni_id,gene_name,ref_seq_id,firstAA_position_in_HELIDR,lastAA_position_in_HELIDR,up_stream_seq,HELIDR_seq,
                          down_stream_seq,Two_S5P_up,Two_S5P_down,'','','','','','','','','']
                df.loc[len(df)] = new_row
            else:
                # We may want to revisit this continue here
                continue
        return df, None
            
    else:
        # print(f'sequence length from structure is {seq_length}, from alpha fold {len(sec_struc_output)}')
        # print("there is an incompatibility between the sequence and alpha fold structure")
        # print(f'the current uniprotid with the issue is: {uni_prot_id}')
        return df, uni_prot_id

## Testing things out

In [10]:
f = open('data/unique_ids_from_spreadsheet.txt', 'r')
f_content = f.readlines()
f.close()

In [11]:
ids = []
for f in f_content:
    ids.append(f.strip())

In [12]:
fail_list = []

In [13]:
def reinitalise_df(annotations = False):
    if annotations:
        df = pd.DataFrame(columns=['uniprot_id','gene_name','refseq_id','firstAA_position_in_HELIDR','lastAA_position_in_HELIDR','HELIDR_upstream_seq'
                       ,'HELIDR_seq', 'HELIDR_downstream_seq', '2S5P_up', '2S5P_down', '2S5P1_up','2S5P1_down','2S5P1_helix','HEK293T_expressed','NonTMD[3]_TMD[2]_SEC[1]',
                       'Non_TMD_classification','4 compartments','TG_CY','TG_SR_nonS', 'annotation'])

    else:
        df = pd.DataFrame(columns=['uniprot_id','gene_name','refseq_id','firstAA_position_in_HELIDR','lastAA_position_in_HELIDR','HELIDR_upstream_seq'
                       ,'HELIDR_seq', 'HELIDR_downstream_seq', '2S5P_up', '2S5P_down', '2S5P1_up','2S5P1_down','2S5P1_helix','HEK293T_expressed','NonTMD[3]_TMD[2]_SEC[1]',
                       'Non_TMD_classification','4 compartments','TG_CY','TG_SR_nonS'])
    return df

In [14]:
counter = 0
df = reinitalise_df(annotations= True)
for i in ids[0:5]:
    if counter%200 == 0:
        print(f'Working on ID: {i} this is entry {counter}/{len(ids)}')
    df, fail = generate_table_rows_for_uniprot_id(i,df, annotate=True)
    if fail != None:
        fail_list.append(fail)
    counter = counter+1
df.to_csv('test.csv')

Working on ID: O00305 this is entry 0/16236


In [16]:
df

,uniprot_id,gene_name,refseq_id,firstAA_position_in_HELIDR,lastAA_position_in_HELIDR,HELIDR_upstream_seq,HELIDR_seq,HELIDR_downstream_seq,2S5P_up,2S5P_down,2S5P1_up,2S5P1_down,2S5P1_helix,HEK293T_expressed,NonTMD[3]_TMD[2]_SEC[1],Non_TMD_classification,4 compartments,TG_CY,TG_SR_nonS,annotation
0,O00305,CACNB4,NM_001320722,61,89,SADSYTSRPS,DSDVSLEEDREAIRQEREQQAAIQLERAK,SKPVAFAVKT,1,0,,,,,,,,,,Region:Disordered;
1,O00305,CACNB4,NM_001320722,154,169,EGCEIGFIPS,PLRLENIRIQQEQKRG,RFHGGKSSGN,0,0,,,,,,,,,,Domain:SH3;
2,O00305,CACNB4,NM_001320722,233,249,VLVGPSLKGY,EVTDMMQKALFDFLKHR,FDGRISITRV,0,0,,,,,,,,,,
3,O00305,CACNB4,NM_001320722,274,280,SLAKRSVLNN,PSKRAII,ERSNTRSSLA,0,0,,,,,,,,,,
4,O00305,CACNB4,NM_001320722,289,302,IIERSNTRSS,LAEVQSEIERIFEL,ARSLQLVVLD,1,0,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Q8WUM4,PDCD6IP,NM_013374,596,639,TALAQDGVIN,EEALSVTELDRVYGGLTTKVQESLKKQEGLLKNIQVSHQEFSKM,KQSNNEANLR,0,0,,,,,,,,,,Region:Interaction with EIAV p9;Region:Interac...
60,Q8WUM4,PDCD6IP,NM_013374,644,713,QEFSKMKQSN,NEANLREEVLKNLATAYDNFVELVANLKEGTKFYNELTEILVRFQN...,AREPSAPSIP,0,0,,,,,,,,,,Region:Interaction with EIAV p9;Region:Interac...
61,Q9H4S2,GSX1,NM_145657,156,168,SSKRMRTAFT,STQLLELEREFAS,NMYLSRLRRI,0,0,,,,,,,,,,
62,Q9H4S2,GSX1,NM_145657,174,183,REFASNMYLS,RLRRIEIATY,LNLSEKQVKI,0,0,,,,,,,,,,


In [153]:
counter = 5000
df = reinitalise_df(True)
for i in ids[5000:10000]:
    if counter%200 == 0:
        print(f'Working on ID: {i} this is entry {counter}/{len(ids)}')
    df, fail = generate_table_rows_for_uniprot_id(i,df, annotate=True)
    if fail != None:
        fail_list.append(fail)
    counter = counter+1
df.to_csv('part_02_annotated.csv')

Working on ID: Q8IX06 this is entry 5000/16236
Working on ID: Q6PL18 this is entry 5200/16236
Working on ID: Q8TD19 this is entry 5400/16236
Working on ID: Q9BZ71 this is entry 5600/16236
Working on ID: Q8IZ08 this is entry 5800/16236
Issue: No gene found found
Working on ID: P35249 this is entry 6000/16236
Issue: No gene found found
Working on ID: A8MQ27 this is entry 6200/16236
Issue: No gene found found
Issue: No gene found found
Working on ID: Q9NX36 this is entry 6400/16236
Working on ID: O75603 this is entry 6600/16236
Working on ID: Q495M3 this is entry 6800/16236
Issue: No gene found found
Working on ID: Q96B01 this is entry 7000/16236
Working on ID: A0A0B4J2F2 this is entry 7200/16236
Working on ID: P49747 this is entry 7400/16236
Issue: No gene found found
Working on ID: Q5JTB6 this is entry 7600/16236
Working on ID: Q9UK96 this is entry 7800/16236
Issue: No gene found found
Working on ID: Q96B26 this is entry 8000/16236
Working on ID: Q8N782 this is entry 8200/16236
Issue: N

In [154]:
counter = 10000
df = reinitalise_df(True)
for i in ids[10000:15000]:
    if counter%200 == 0:
        print(f'Working on ID: {i} this is entry {counter}/{len(ids)}')
    df, fail = generate_table_rows_for_uniprot_id(i,df, annotate=True)
    if fail != None:
        fail_list.append(fail)
    counter = counter+1
df.to_csv('part_03_annotated.csv')

Working on ID: Q8N5Y2 this is entry 10000/16236
Issue: No gene found found
Issue: No gene found found
Working on ID: Q711Q0 this is entry 10200/16236
Working on ID: Q9H063 this is entry 10400/16236
Working on ID: Q9HCK1 this is entry 10600/16236
Working on ID: O60774 this is entry 10800/16236
Working on ID: Q8WUK0 this is entry 11000/16236
Working on ID: P0DP08 this is entry 11200/16236
Working on ID: Q86V35 this is entry 11400/16236
Working on ID: O15013 this is entry 11600/16236
Working on ID: Q8NGG8 this is entry 11800/16236
Issue: No gene found found
Issue: No gene found found
Working on ID: Q9P2M7 this is entry 12000/16236
Working on ID: P01584 this is entry 12200/16236
Working on ID: Q6PKG0 this is entry 12400/16236
Working on ID: Q08477 this is entry 12600/16236
Working on ID: Q3SYA9 this is entry 12800/16236
Working on ID: P25789 this is entry 13000/16236
Working on ID: A0AV96 this is entry 13200/16236
Working on ID: Q6PH85 this is entry 13400/16236
Working on ID: Q8NHH9 this i

In [155]:
counter = 15000
df = reinitalise_df(True)
for i in ids[15000:]:
    if counter%200 == 0:
        print(f'Working on ID: {i} this is entry {counter}/{len(ids)}')
    df, fail = generate_table_rows_for_uniprot_id(i,df,annotate=True)
    if fail != None:
        fail_list.append(fail)
    counter = counter+1
df.to_csv('part_04_annotated.csv')

Working on ID: P52294 this is entry 15000/16236
Working on ID: Q4G0T1 this is entry 15200/16236
Working on ID: Q9NPH3 this is entry 15400/16236
Issue: No gene found found
Working on ID: Q9BTL3 this is entry 15600/16236
Working on ID: Q9H8M1 this is entry 15800/16236
Working on ID: Q6VMQ6 this is entry 16000/16236
Working on ID: Q5VSL9 this is entry 16200/16236


In [121]:
print(fail_list)
np.savetxt('data/failed_AF_structures.csv', fail_list, fmt='%s')

['Q3KNS1', 'Q96MI6', 'Q99102', 'Q6ZR37', 'P36639', 'Q12983', 'Q9NX55', 'P47756', 'Q96AX9', 'A6NMZ5', 'Q5H9F3', 'Q3ZCN5', 'B4E2M5', 'O43451', 'O43309', 'P17019', 'P0DN82', 'Q13507', 'P02708', 'P20848', 'P26599', 'Q9BV97', 'Q8NDH2', 'Q9BRI3', 'Q8NGU1', 'P23109', 'Q2VWA4', 'Q16635', 'Q6NY19', 'Q8N859', 'Q92782', 'P59826', 'Q6ZTN6', 'P0DMP1', 'Q8N7C3', 'Q6P3W6', 'O15131', 'Q9UPX8', 'Q6ZP01', 'Q8NFD5', 'Q7Z7G0', 'O94851', 'A6NNF4', 'Q8NH41', 'P0C623', 'Q9HD15', 'Q8IZM8', 'O15016', 'Q8N782', 'Q8IV03', 'Q9H310', 'Q03518', 'Q66PJ3', 'Q8TAX9', 'Q8N423', 'O94901', 'Q9HBL0', 'Q3I5F7', 'Q96Q27', 'Q9BY31', 'Q15170', 'Q9UJX3', 'Q12766', 'D6RF30', 'P29973', 'A0A0U1RRI6', 'Q0D2K0', 'Q9P0K9', 'Q9P2M7', 'Q96KE9', 'Q15270', 'A8MZ97', 'P62861', 'Q9Y5I7', 'Q9HBW0', 'Q5XKL5', 'Q53TS8', 'O94880', 'Q16850', 'Q6ZMS7', 'Q8NBS3', 'Q6ZMR5', 'P51606', 'Q9NRJ5', 'Q96FT7', 'Q6PKH6']


## Playground testing things out

In [80]:
domain_filter = []
f = open('domain.csv')
f_content = f.readlines()
f.close()

for l in f_content:
    domain_filter.append(l.strip())


In [83]:
region_filter = []
f = open('region_filter.csv')
f_content = f.readlines()
f.close()

for l in f_content:
    region_filter.append(l.strip())

In [87]:
discard_list = np.hstack((region_filter,domain_filter))

In [89]:
discard_list[0]

'ARM-like and Heat-like helical repeats'

In [84]:
uni_prot_id = 'Q9H4S2'
work_dir = 'temp'
print('Working on uniprot ID', uni_prot_id)
# Downloading uniprot file and alphafold file
download_uniprot_json_file(uni_prot_id, work_dir)
download_alpha_fold_pdbs([uni_prot_id], workdir =work_dir)
p = 'temp/AF-'+uni_prot_id+'-F1-model_v2.pdb'
sec_struc_output = get_sec_struct(p, '/Users/toni_brain/miniconda3/envs/dssp//bin/mkdssp')
print('got secondary structure')
print(sec_struc_output)

# Process uniprot file
f = open('temp/'+uni_prot_id+'.json')
data = json.load(f)
f.close()

print('read json file successfully')

Working on uniprot ID Q9H4S2
At entry 0/1
ID: Q9H4S2
got secondary structure
['-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' 'T' 'T' '-' '-' 'T' 'T' 'S'
 '-' '-' '-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'T' 'T' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' 'T' 'T' 'S' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'H' 'H' 'H' 'H' 'H' 'H' '-' 'S' 'S' '-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'H' 'H' 'H' 'T' 'T' '-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'T' 'T' 'T' 'T' 'T' 'T' '-' '-' 'S' '-' '-'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-

In [85]:
sec_struc_output

array(['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', 'T', 'T', '-', '-', 'T',
       'T', 'S', '-', '-', '-', '-', 'H', 'H', 'H', 'H', 'H', 'H', 'T',
       'T', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'T', 'T',
       'S', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-',
       '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'H',
       'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', '-',
       'S', 'S', '-', '-', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H

In [86]:
# This is collecting known information

known_things = {}
known_things['Domain'] = []
known_things['Region'] = []
known_things['Motif'] = []
#known_things['Helix'] = []

for d in data['features']:
    #print(d['type'])
    if d['type'] == 'Domain':
        known_things['Domain'].append([d['location']['start']['value'], d['location']['end']['value'], d['description']])
    elif d['type'] == 'Region':
        known_things['Region'].append([d['location']['start']['value'], d['location']['end']['value'], d['description']])
    elif d['type'] == 'Motif':
        known_things['Motif'].append([d['location']['start']['value'], d['location']['end']['value'], d['description']])
    #elif d['type'] == 'Helix':
    #    known_things['Helix'].append([d['location']['start']['value'], d['location']['end']['value'], d['description']])

In [87]:
known_things

{'Domain': [],
 'Region': [[1, 20, 'SNAG domain'], [201, 264, 'Disordered']],
 'Motif': []}

In [88]:
for d in data['features']:
    if d['type']=='Domain':
        print (d)
    if d['type']=='Region':
        print (d)

{'type': 'Region', 'location': {'start': {'value': 1, 'modifier': 'EXACT'}, 'end': {'value': 20, 'modifier': 'EXACT'}}, 'description': 'SNAG domain', 'evidences': [{'evidenceCode': 'ECO:0000250'}]}
{'type': 'Region', 'location': {'start': {'value': 201, 'modifier': 'EXACT'}, 'end': {'value': 264, 'modifier': 'EXACT'}}, 'description': 'Disordered', 'evidences': [{'evidenceCode': 'ECO:0000256', 'source': 'SAM', 'id': 'MobiDB-lite'}]}


In [18]:
sequence_length = data['sequence']['length']
if sequence_length == len(sec_struc_output):
    print('yes')
    bool_array =  np.zeros(sequence_length, dtype=bool)

yes


In [45]:
bool_array =  np.zeros(sequence_length, dtype=bool)
discard_list = ['bHLH', 'Leucine-zipper']
alpha_helix_index_list = get_alpha_helix_length_and_location(sec_struc_output, min_length=7)
helix_list_remove_index = []
for key in known_things.keys():
    for entry in known_things[key]:
        if entry[-1] in discard_list:
            bool_array[entry[0]-1:entry[1]] = True
            index = 0
            for a in alpha_helix_index_list:
                curr_helix =  np.zeros(sequence_length, dtype=bool)
                curr_helix[a] = True
                overalp = (np.sum(np.logical_and(bool_array, curr_helix)))
                if overalp>4:
                    helix_list_remove_index.append(index)
                index = index+1
unique_idx = np.unique(helix_list_remove_index)
for index in sorted(unique_idx, reverse=True):
    del alpha_helix_index_list[index]
print (alpha_helix_index_list)

[[94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106], [121, 122, 123, 124, 125, 126, 127, 128, 129, 130], [139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153], [175, 176, 177, 178, 179, 180, 181, 182, 183]]
